<a href="https://colab.research.google.com/github/javid321/GitPullTest/blob/master/Klasifikasi_Video_Deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Demo Aplikasi Hasil Skripsi Javid Al Haq

In [ ]:
# https://machinelearningmastery.com/how-to-perform-face-detection-with-classical-and-deep-learning-methods-in-python-with-keras/

In [ ]:
!pip install mtcnn
!pip show mtcnn

import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from collections import deque
print("tf",tf.__version__)
import glob
from tensorflow import keras
from tensorflow.keras.applications.xception import preprocess_input as pix
from tensorflow.keras.applications.resnet50 import preprocess_input as pir
from keras.preprocessing.image import load_img
from skimage.feature import local_binary_pattern
import time
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow
from google.colab import drive

import csv
import datetime
from matplotlib import pyplot
from pathlib import Path
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN
# confirm mtcnn was installed correctly
import mtcnn
# Import functions and libraries

%matplotlib inline
# print version
print(mtcnn.__version__)
from tqdm import tqdm

     |████████████████████████████████| 2.3 MB 5.3 MB/s 
Name: mtcnn
Version: 0.1.1
Summary: Multi-task Cascaded Convolutional Neural Networks for Face Detection, based on TensorFlow
Home-page: http://github.com/ipazc/mtcnn
Author: Iván de Paz Centeno
Author-email: ipazc@unileon.es
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: keras, opencv-python
Required-by: 
tf 2.7.0
0.1.0


Sambungkan ke GDrive

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


Fungsi LBP

In [ ]:
def lbp_(image, n_points, radius):
  
    # compute the Local Binary Pattern representation
    # of the image, and then use the LBP representation
    # to build the histogram of patterns
    lbp = local_binary_pattern(image, n_points, radius,  method='default')
    return lbp

Fungsi Pemrosesan Gambar 

In [ ]:
def process(imag) :
    # settings for LBP
    radius = 2
    n_points = 8 * radius
    imgg = cv.GaussianBlur(imag,(5,5),2)
    _img = imgg

    for ichannel in range(_img.shape[2]):
       _img[:,:,ichannel] = lbp_(imgg[:,:,ichannel],n_points,radius)
        
    imggray = cv.cvtColor(_img, cv.COLOR_BGR2GRAY)
    return imggray

Label

In [ ]:
label = { 0 : "deepfake", 1 : "real"}
print("Label : ", label) #label kelas

Label :  {0: 'deepfake', 1: 'real'}


Load Model

In [ ]:
load_model = tf.keras.models.load_model('drive/My Drive/model/_Xception20E_FA2.h5') # load model dari Google Drive

Preprocessing Untuk Input Model Xception

In [ ]:
def prepare(filepath):
    img = keras.preprocessing.image.load_img(
    filepath, target_size= (224,224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array.reshape((1, img_array.shape[0], img_array.shape[1], img_array.shape[2])) 
    img_array = pix(img_array)
    return img_array

Preprocessing Untuk Input Model ResNet-50

In [ ]:
def preparer(filepath):
    img = keras.preprocessing.image.load_img(
    filepath, target_size= (224,224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array.reshape((1, img_array.shape[0], img_array.shape[1], img_array.shape[2])) 
    img_array = pir(img_array)
    return img_array

Path Simpan

In [ ]:
frames_path = os.path.join(
    "drive/My Drive/deteksi/DeteksiF/frame") # penyimpanan frame 
wajah_path = os.path.join(
    "drive/My Drive/deteksi/DeteksiF/ori") # penyimpanan wajah 
wajah_path_pre = os.path.join(
    "drive/My Drive/deteksi/DeteksiF/prepro") # penyimpanan wajah hasil preprocess
vid_path = os.path.join("drive/My Drive/deteksi/video_pred") # penyimpanan video hasil klasifikasi
my_file = Path("drive/My Drive/databaru/datatesreal.csv") # penyimpanan metadata percobaan
my_file_ = Path("drive/My Drive/databaru/datatesreal.csv") # penyimpanan metadata percobaan

Fungsi Klasifikasi 1 Video

In [ ]:
def klas_vid (vid):
      # create the detector, using default weights
      detector = MTCNN()

      x=1
      k=1
  
      _video = cv.VideoCapture(vid)      
      print('video =', vid)
      print("Processing.....")

      start_time = time.time()
      writer = None
      filenamepre = None
      name = os.path.basename(vid)
      name = name.replace(name[-4:],"")
      m=0
      fps = _video.get(cv.CAP_PROP_FPS)
      max = 0
      maxlabel = 0.0
      Q = deque(maxlen=3)
      print("fps =",fps)

      while (_video):
          
          success,image = _video.read()

          if (success == False):
                if k==1 :
                  print("Gagal")
                  break;
                break;

          count = 1
         
          # detect faces in the image
          faces = detector.detect_faces(image)
          
          for i in faces :
              x1, y1, w, h = i['box']
              x2, y2 = x1 + w, y1 + h
              if (count > 1) :
                    break;
                          
              try:
                framename = os.path.join(frames_path, "frame_" + str(k) + ".jpg")
                cv.imwrite(framename,image)
                shape = [(x1, y1), (x2, y2)]
                # create  rectangleimage
                img_box = Image.open(framename)
                img_draw = ImageDraw.Draw(img_box)  
                img_draw.rectangle(shape, outline='red', width = 3)
                img_box = img_box.save(framename)

                image_w = cv.resize(image[y1:y2, x1:x2],
                                  (224,224), interpolation = cv.INTER_AREA)
                filename = os.path.join(wajah_path, "wajah_" + str(k) + ".jpg")
                cv.imwrite(filename,image_w)    

                count+=1

                image_w = cv.imread(filename)
                image_pre = process(image_w)
                filenamepre = os.path.join(
                        wajah_path_pre, "wajah_LBP_" + str(k) + ".jpg")
                plt.imsave(filenamepre,image_pre,cmap = 'gray')
                k +=1
              except cv.error as e:
                print('Invalid frame!')

          imgbox_ = cv.imread(framename)
          frame_ = imgbox_.copy()  
          if filenamepre != None :
              fr = os.path.basename(framename)
              fr = fr[-5:-4]
              pr = os.path.basename(filenamepre)
              pr = pr[-5:-4]
              if fr == pr :
                data_path = filenamepre
                prediction = load_model.predict([prepare(data_path)])
                score = (prediction[0])
                max = max + (100 * np.max(score))
                m +=1
                maxlabel = maxlabel + score
                # draw the activity on the output frame
                Q.append(score)
                # perform prediction averaging over the current history of
                # previous predictions
                results = np.array(Q).mean(axis=0)
                i = np.argmax(results)
                lab_ = label[i]
                pred = ("{}" .format(lab_))
                cv.putText(frame_, pred, (35, 50), cv.FONT_HERSHEY_SIMPLEX, 1.25, (0,0,255), 5)
          vid_name = os.path.join(vid_path, "klsf_"+name+".mp4")
          h,w,_= image.shape
          size = (w, h)
          # check if the video writer is None
          if writer is None:
              # initialize our video writer
              fourcc = cv.VideoWriter_fourcc(*"MJPG")
              writer = cv.VideoWriter(vid_name, fourcc, fps, size, True)
          # write the output frame to disk
          writer.write(frame_)
          if k == 6 :
            break;
      writer.release()
      _video.release()
      total_frame = k-1
      waktu = time.time() - start_time
      print(" ")
      print("total frame :",total_frame)
      print("Waktu Proses :", waktu, " detik")
      vide_name = os.path.join(vid_path, "klsf_{}_{:.2f}%_"
                    .format(label[np.argmax(maxlabel/m)], max/m)+name+".mp4")
      os.rename(vid_name,vide_name)
      print("Disimpan :", vide_name)
      print('Proses Menyimpan.....')
      for i in tqdm(range(0,140)) :
          time.sleep(1)
          if i == 139 :
            print("\nselesai")

Fungsi Klasifikasi Banyak Video

In [ ]:
def klass_vid (vid):
      # create the detector, using default weights
      detector = MTCNN()

      x=1
      k=1
  
      _video = cv.VideoCapture(vid)      
      print('video =', vid)
      print("Processing.....")

      start_time = time.time()
      writer = None
      filenamepre = None
      name = os.path.basename(vid)
      name = name.replace(name[-4:],"")
      m=0
      fps = _video.get(cv.CAP_PROP_FPS)
      max = 0
      maxlabel = 0.0
      Q = deque(maxlen=3)
      print("fps =",fps)

      while (_video):
          
          success,image = _video.read()

          if (success == False):
                if k==1 :
                  print("Gagal")
                  break;
                break;

          count = 1
         
          # detect faces in the image
          faces = detector.detect_faces(image)
          
          for i in faces :
              x1, y1, w, h = i['box']
              x2, y2 = x1 + w, y1 + h
              if (count > 1) :
                    break;
                          
              try:
                framename = os.path.join(frames_path, "frame_" + str(k) + ".jpg")
                cv.imwrite(framename,image)
                shape = [(x1, y1), (x2, y2)]
                # create  rectangleimage
                img_box = Image.open(framename)
                img_draw = ImageDraw.Draw(img_box)  
                img_draw.rectangle(shape, outline='red', width = 3)
                img_box = img_box.save(framename)

                image_w = cv.resize(image[y1:y2, x1:x2],
                                  (224,224), interpolation = cv.INTER_AREA)
                filename = os.path.join(wajah_path, "wajah_" + str(k) + ".jpg")
                cv.imwrite(filename,image_w)    

                count+=1

                image_w = cv.imread(filename)
                image_pre = process(image_w)
                filenamepre = os.path.join(
                        wajah_path_pre, "wajah_LBP_" + str(k) + ".jpg")
                plt.imsave(filenamepre,image_pre,cmap = 'gray')
                k +=1
              except cv.error as e:
                print('Invalid frame!')

          imgbox_ = cv.imread(framename)
          frame_ = imgbox_.copy()  
          if filenamepre != None :
              fr = os.path.basename(framename)
              fr = fr[-5:-4]
              pr = os.path.basename(filenamepre)
              pr = pr[-5:-4]
              if fr == pr :
                data_path = filenamepre
                prediction = load_model.predict([prepare(data_path)])
                score = (prediction[0])
                max = max + (100 * np.max(score))
                m +=1
                maxlabel = maxlabel + score
                # draw the activity on the output frame
                Q.append(score)
                # perform prediction averaging over the current history of
                # previous predictions
                results = np.array(Q).mean(axis=0)
                i = np.argmax(results)
                lab_ = label[i]
                pred = ("{}" .format(lab_))
                cv.putText(frame_, pred, (35, 50), cv.FONT_HERSHEY_SIMPLEX, 1.25, (0,0,255), 5)
          vid_name = os.path.join(vid_path, "klsf_"+name+".mp4")
          h,w,_= image.shape
          size = (w, h)
          # check if the video writer is None
          if writer is None:
              # initialize our video writer
              fourcc = cv.VideoWriter_fourcc(*"MJPG")
              writer = cv.VideoWriter(vid_name, fourcc, fps, size, True)
          # write the output frame to disk
          writer.write(frame_)
          # if k == 6 : #pembatasan jumlah frame
          #   break;
      writer.release()
      _video.release()
      total_frame = k-1
      waktu = time.time() - start_time
      print(" ")
      print("total frame :",total_frame)
      print("Waktu Proses :", waktu, " detik")
      vide_name = os.path.join(vid_path, "klsf_{}_{:.2f}%_"
                    .format(label[np.argmax(maxlabel/m)], max/m)+name+".mp4")
      os.rename(vid_name,vide_name)
      print("Disimpan :", vide_name)
      print('Proses Menyimpan.....')
      
      tgl = datetime.datetime.now()
      tgl = tgl.strftime("%x")
      print(tgl)

      filename = os.path.basename(vid)
      simpanname = os.path.basename(vide_name)
      simpanname = simpanname.replace(filename,"")
      simpanname = simpanname.replace("klsf_","")
      with open('drive/My Drive/databaru/datatesreal.csv', 'a', newline = '') as file:
        writer = csv.writer(file)
        writer.writerow([filename, total_frame, waktu, simpanname, tgl])
        file.close()
      for i in tqdm(range(0,5)) :
          time.sleep(1)
          if i == 4 :
            print("\nselesai")

File Video Yang Akan Diklasifikasi # 1 Video

In [ ]:
# #video_ = glob.glob("drive/My Drive/datasetdeepfake/Celeb-real/*.mp4")
# video_ = glob.glob("drive/My Drive/datasetdeepfake/Celeb-synthesis/*.mp4")

# v = 0
# for get in video_ :
#     if v == 350 :
#         get = get
#         get_vid = get
#         break;
#     v +=1
# print(get_vid)

#file = "drive/My Drive/datasetdeepfake/Celeb-synthesis/id0_id1_0003.mp4" # nama file video


Tentukan Video Yang akan Diklasifikasi # 1 Video

In [ ]:
# get_vid = "drive/My Drive/deepfake/TrumpBeanDeepfake.mp4"
# os.path.isfile(get_vid) 
# print(get_vid)

Klasifikasi 1 Video

In [ ]:
#klasifikasi = klas_vid(get_vid)

Klasifikasi Banyak Video

In [ ]:

video_ = glob.glob("drive/My Drive/datasetdeepfake/Celeb-real/*.mp4")
# video_ = glob.glob("drive/My Drive/datasetdeepfake/Celeb-synthesis/*.mp4")

v = 0

if my_file.is_file():
  f = open(my_file, "w+")
  with open('drive/My Drive/databaru/datatesreal.csv', 'a', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Nama", "Total Frame", "Waktu", "Hasil", "Tanggal"])
  f.close()
  pass;
else :
  with open('drive/My Drive/databaru/datatesreal.csv', 'w', newline = '') as file:
      writer = csv.writer(file)
      writer.writerow(["Nama", "Total Frame", "Waktu", "Hasil", "Tanggal"])


for get in video_ :
  if v >= 350 :
    get = get
    get_vid = get
    print("video ke :",v,":",get_vid)
    klasifikasi = klass_vid(get_vid)

    if v == 359 :
      break;
  v +=1
  

video ke : 350 : drive/My Drive/datasetdeepfake/Celeb-real/id41_0008.mp4
video = drive/My Drive/datasetdeepfake/Celeb-real/id41_0008.mp4
Processing.....
fps = 30.0
 
total frame : 329
Waktu Proses : 551.9218606948853  detik
Disimpan : drive/My Drive/deteksi/video_pred/klsf_real_71.77%_id41_0008.mp4
Proses Menyimpan.....
12/02/21


100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


selesai
video ke : 351 : drive/My Drive/datasetdeepfake/Celeb-real/id51_0004.mp4


video = drive/My Drive/datasetdeepfake/Celeb-real/id51_0004.mp4
Processing.....
fps = 29.000007929408085
 
total frame : 447
Waktu Proses : 573.108074426651  detik
Disimpan : drive/My Drive/deteksi/video_pred/klsf_real_79.26%_id51_0004.mp4
Proses Menyimpan.....
12/02/21


100%|██████████| 5/5 [00:05<00:00,  1.00s/it]

...

100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


selesai
video ke : 359 : drive/My Drive/datasetdeepfake/Celeb-real/id51_0009.mp4


video = drive/My Drive/datasetdeepfake/Celeb-real/id51_0009.mp4
Processing.....
fps = 30.0
 
total frame : 512
Waktu Proses : 510.9406313896179  detik
Disimpan : drive/My Drive/deteksi/video_pred/klsf_real_67.09%_id51_0009.mp4
Proses Menyimpan.....
12/02/21


100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


selesai


In [ ]:
 import pandas as pd

pd.read_csv("drive/My Drive/databaru/datatesdeepfakeResNet-50.csv",usecols=["Nama", "Total Frame", "Waktu", "Hasil Rata2", "Tanggal"])

,Nama,Total Frame,Waktu,Hasil Rata2,Tanggal
0,id21_id25_0008.mp4,322,738.725894,deepfake_66.96%_,12/01/21
1,id53_id50_0005.mp4,466,642.369837,deepfake_73.60%_,12/01/21
2,id31_id26_0006.mp4,365,357.471821,deepfake_77.81%_,12/01/21
3,id53_id56_0009.mp4,451,432.906776,real_72.69%_,12/01/21
4,id37_id4_0007.mp4,473,466.230234,real_65.68%_,12/01/21
5,id3_id37_0007.mp4,361,344.741217,real_71.76%_,12/01/21
6,id0_id2_0008.mp4,464,451.561153,deepfake_86.23%_,12/01/21
7,id41_id47_0008.mp4,329,317.516426,deepfake_68.12%_,12/01/21
8,id50_id55_0007.mp4,437,424.043643,real_67.32%_,12/01/21
9,id1_id30_0004.mp4,409,350.732740,deepfake_71.65%_,12/01/21
